In [2]:
import hashlib
import json
import datetime

# ==========================================
# 1. ALTYAPI VE YARDIMCI FONKSİYONLAR
# ==========================================

# Blockchain Defterini temsil eden liste (Veritabanı)
BLOCKCHAIN_DB = []

def hash_olustur(veri):
    """Verinin SHA-256 dijital parmak izini oluşturur."""
    # Veriyi standart bir JSON formatına çevirip hash'liyoruz
    veri_str = json.dumps(veri, sort_keys=True)
    return hashlib.sha256(veri_str.encode('utf-8')).hexdigest()

def kriptografik_imza_simulasyonu(veri_hash, ozel_anahtar):
    """
    Kullanıcının Özel Anahtarı ile veriyi imzalamasını simüle eder.
    Gerçek dünyada RSA/ECDSA kullanılır, burada mantığı göstermek için hash birleştirme yapılmaktadır.
    """
    imza_birlestirme = veri_hash + ozel_anahtar
    return hash_olustur(imza_birlestirme)

# ==========================================
# 2. AKILLI SÖZLEŞME SINIFI 
# ==========================================

class ArabuluculukAkilliSozlesmesi:
    def __init__(self, dosya_no, anlasma_metni, anlasma_turu):
        self.dosya_no = dosya_no
        self.anlasma_metni = anlasma_metni
        self.anlasma_turu = anlasma_turu # Örn: 'Sigorta_Anlasma', 'Para_Borcu','Arabuluculuk_Son_Tutanagi
        
        # Adım 2: Belge Bütünlüğü (Hash Oluşturma)
        self.tutanak_hash = hash_olustur(anlasma_metni)
        
        self.imzalar = {}
        self.tescil_durumu = False # Başlangıçta tescil edilmemiş
        self.resmi_onay_kodu = None # Mahkeme şerhi
        
        print(f"📄 [{self.dosya_no}] Sözleşme Oluşturuldu.")
        print(f"🔒 Tutanak Hash Değeri: {self.tutanak_hash}\n")

    def imza_ekle(self, taraf_adi, imza_degeri, taraf_ozel_anahtar_kontrolu):
        """
        Adım 2 & 3: İmzaları toplar, doğrular ve 3 imza tamamlanınca Tescil eder.
        """
        # İmza Doğrulama (Public Key kontrolü simülasyonu)
        beklenen_imza = kriptografik_imza_simulasyonu(self.tutanak_hash, taraf_ozel_anahtar_kontrolu)
        
        if imza_degeri != beklenen_imza:
            print(f"❌ HATA: {taraf_adi} için geçersiz imza! İşlem reddedildi.")
            return False
            
        self.imzalar[taraf_adi] = imza_degeri
        print(f"✅ {taraf_adi} imzası doğrulandı ve eklendi.")
        
        # 3 İmza tamamlandı mı kontrol et
        if len(self.imzalar) == 3:
            self.blok_zincirine_kaydet()
            
        return True

    def blok_zincirine_kaydet(self):
        """Adım 3: Tescil İşlemi (Değiştirilemez Kayıt)"""
        self.tescil_durumu = True
        
        yeni_blok = {
            "zaman_damgasi": str(datetime.datetime.now()),
            "dosya_no": self.dosya_no,
            "tutanak_hash": self.tutanak_hash,
            "imzacilar": list(self.imzalar.keys()),
            "durum": "TESCİL EDİLDİ - İLAM NİTELİĞİNDE BELGE"
        }
        
        BLOCKCHAIN_DB.append(yeni_blok)
        print("\n TEBRİKLER! Tüm imzalar tamamlandı.")
        print(" Anlaşma Blok Zincirine Değiştirilemez Şekilde Kaydedildi.")
        print("-" * 50)

    def icra_islemini_baslat(self, mahkeme_kodu):
        """
        Adım 4: Yarı-Otomatik İcra (Resmi Doğrulama)
        Oracle yerine UYAP/e-Devlet entegrasyonu simülasyonu.
        """
        print(f"\n🏛️  Mahkeme Onay Kodu (Şerh) Girildi: {mahkeme_kodu}")
        print("🔄 Sistem UYAP Veritabanını Sorguluyor...")
        
        if not self.tescil_durumu:
            print("❌ HATA: Sözleşme henüz tescil edilmemiş.")
            return

        # UYAP Simülasyonu: Beklenen kod "TC_MAHKEME_2025_OK" olsun
        BEKLENEN_RESMI_KOD = "TC_MAHKEME_2025_OK"
        
        if mahkeme_kodu == BEKLENEN_RESMI_KOD:
            self.resmi_onay_kodu = mahkeme_kodu
            print("✅ ONAYLANDI: Mahkeme Şerhi Doğrulandı.")
            
            if self.anlasma_turu == "Tasinmaz_Devri":
                print("🚀 İŞLEM TETİKLENDİ: Tapu Müdürlüğü Sistemine 'Satış Yetkisi' emri gönderildi.")
            elif self.anlasma_turu == "Para_Borcu":
                print("🚀 İŞLEM TETİKLENDİ: Banka Sistemine 'Ödeme Emri' gönderildi.")
        else:
            print("⛔ REDDEDİLDİ: Girilen kod UYAP kayıtlarıyla eşleşmiyor.")

# ==========================================
# 3. SİMÜLASYONU ÇALIŞTIRMA (DEMO)
# ==========================================

# --- A. Verilerin Hazırlanması (Adım 1) ---
# Temsili Özel Anahtarlar (Adım 1'de üretilenler)
ANAHTAR_A = "ozel_anahtar_taraf_A_123"
ANAHTAR_B = "ozel_anahtar_taraf_B_456"
ANAHTAR_M = "ozel_anahtar_arabulucu_789"

tutanak_metni = "Taraflara arabuluculuğun temel ilkeleri, arabuluculuk süreci ve arabuluculuk süreci sonunda hazırlanan arabuluculuk son tutanağının hukuki ve mali yönlerden bütün sonuçları hakkında bilgi verildi.Taraflar söz alarak arabuluculuğun temel ilkelerini, arabuluculuk sürecini ve arabuluculuk süreci sonunda hazırlanan arabuluculuk son tutanağının hukuki ve mali yönlerden bütün sonuçlarını anladık dediler.Taraflar müzakereler sonucunda Taraf A'nın Taraf B'den kıdem tazminatı alacağının 20.01.2026 tarihinde 280.000 TL bedelle  ödenmesi hususunda anlaşmaya varmışlardır. Taraflar üzerinde anlaşılan hususlarda dava açılamayacağını anladıklarını kabul ve beyan ettiler son tutanağa böyle geçsin dediler.İşbu arabuluculuk son tutanağı iki sayfa ve dört nüsha olarak 6325 sayılı Hukuk Uyuşmazlıklarında Arabuluculuk Kanunu m. 17 ve 7036 sayılı İş Mahkemeleri Kanunu m. 3 uyarınca hep birlikte imza altına alındı."
dosya_numarasi = "ARB-2025/001"

# --- B. Sözleşmenin Başlatılması ---
kontrat = ArabuluculukAkilliSozlesmesi(dosya_numarasi, tutanak_metni, "Kıdem_Tazminati")

# --- C. İmzalama Süreci (Adım 2) ---
# Taraflar hash'i kendi anahtarlarıyla imzalıyor
imza_a_degeri = kriptografik_imza_simulasyonu(kontrat.tutanak_hash, ANAHTAR_A)
imza_b_degeri = kriptografik_imza_simulasyonu(kontrat.tutanak_hash, ANAHTAR_B)
imza_m_degeri = kriptografik_imza_simulasyonu(kontrat.tutanak_hash, ANAHTAR_M)

# İmzaların Sisteme Gönderilmesi
kontrat.imza_ekle("Taraf A", imza_a_degeri, ANAHTAR_A)
kontrat.imza_ekle("Taraf B", imza_b_degeri, ANAHTAR_B)
kontrat.imza_ekle("Arabulucu", imza_m_degeri, ANAHTAR_M) # Bu işlem TESCİL'i tetikler

# --- D. İcra Aşaması (Adım 4) ---
# Senaryo 1: Yanlış Kod Denemesi
kontrat.icra_islemini_baslat("YANLIS_KOD_123")

# Senaryo 2: Doğru Kod (İcra Edilebilirlik Şerhi)
kontrat.icra_islemini_baslat("TC_MAHKEME_2025_OK")

📄 [ARB-2025/001] Sözleşme Oluşturuldu.
🔒 Tutanak Hash Değeri: e3613dd80891fdc9187a7ecfb15459992655ad289196942233424f2e4e260f4d

✅ Taraf A imzası doğrulandı ve eklendi.
✅ Taraf B imzası doğrulandı ve eklendi.
✅ Arabulucu imzası doğrulandı ve eklendi.

 TEBRİKLER! Tüm imzalar tamamlandı.
 Anlaşma Blok Zincirine Değiştirilemez Şekilde Kaydedildi.
--------------------------------------------------

🏛️  Mahkeme Onay Kodu (Şerh) Girildi: YANLIS_KOD_123
🔄 Sistem UYAP Veritabanını Sorguluyor...
⛔ REDDEDİLDİ: Girilen kod UYAP kayıtlarıyla eşleşmiyor.

🏛️  Mahkeme Onay Kodu (Şerh) Girildi: TC_MAHKEME_2025_OK
🔄 Sistem UYAP Veritabanını Sorguluyor...
✅ ONAYLANDI: Mahkeme Şerhi Doğrulandı.
